In [2]:
import pandas as pd
from datasets import load_dataset

In [15]:
import os
path='data/processed'
if not os.path.exists(path):
    os.makedirs(path)

#  Multilingual Amazon Reviews Corpus

To make the implementation easy we are using huggingface dataset for the dataset. One can also download the dataset from original site

In [6]:
amaz_t={
    'de':'Rezension: {s} Bewertung:',
    'ja':'レビュー: {s} 評価:',
    'es':'Revisar: {s} Clasificación:',
    'fr':'Examen: {s} Évaluation:',
    'zh':'审查: {s} 评分:',
    'en':'Review: {s} Rating:'
    
}

amaz_bi_v={
    'en':{
        'negative':'bad',
        'positive':'good'
    },
    'fr':{
        'negative':'mal',
        'positive':'bien'
    },
    'es':{
        'negative':'malo',
        'positive':'bueno'
    },
    'ja':{
        'negative':'悪い',
        'positive':'良い'
    },
    'zh':{
        'negative':'坏的',
        'positive':'好的'
    },
    'de':{
        'negative':'Schlecht',
        'positive':'gut'
    }
}

In [18]:
# train set
train_marc=load_dataset('amazon_reviews_multi','all_languages')['train']

language=[]
input=[]
output=[]
id=[]

for d in train_marc:

    l=d['language']

    if d['stars']==3:
        continue
    elif d['stars'] in [1,2]:
        output.append(amaz_bi_v[l]['negative'])
    elif d['stars'] in [4,5]:
        output.append(amaz_bi_v[l]['positive'])
    
    input.append(amaz_t[l].format(s=d['review_body']))
    id.append(d['review_id'])
    language.append(l)

marc_df={
    'id':id,
    'input':input,
    'output':output,
    'language':language
}
marc_df=pd.DataFrame(marc_df)
marc_df.to_csv('data/processed/amaz_bi_train.csv')

Reusing dataset amazon_reviews_multi (/home/eshaant/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
# test set
test_marc=load_dataset('amazon_reviews_multi','all_languages')['test']

language=[]
input=[]
output=[]
id=[]

for d in test_marc:

    l=d['language']

    if d['stars']==3:
        continue
    elif d['stars'] in [1,2]:
        output.append(amaz_bi_v[l]['negative'])
    elif d['stars'] in [4,5]:
        output.append(amaz_bi_v[l]['positive'])
    
    input.append(amaz_t[l].format(s=d['review_body']))
    id.append(d['review_id'])
    language.append(l)

marc_df={
    'id':id,
    'input':input,
    'output':output,
    'language':language
}
marc_df=pd.DataFrame(marc_df)
marc_df.to_csv('data/processed/amaz_bi_test.csv')


Reusing dataset amazon_reviews_multi (/home/eshaant/.cache/huggingface/datasets/amazon_reviews_multi/all_languages/1.0.0/724e94f4b0c6c405ce7e476a6c5ef4f87db30799ad49f765094cf9770e0f7609)


  0%|          | 0/3 [00:00<?, ?it/s]

# Hateval 2019

INSTRUCTION FOR HATE EVAL

We have used relabeled version of Hateeval that can be found in [this link](https://github.com/sentropytechnologies/hateval2019-relabeled)
please download the dataset from the said link and save the csvs in a folder data/raw/hateval2019

The git goesnt contain text(tweets), so to extract that download the original hateval by downloading from [this link](http://hatespeech.di.unito.it/hateval.html) and save the csvs in a folder data/raw/hateval2019/original

We have included the validation set of hateval in our traning set to increase the data retrival space

In [78]:
hateval_t={
    'es':'correo: {s} Es odioso?',
    'en':'Post: {s} Is hatefull:'
    
}

hateval_v={
    'en':{
        1:'yes',
        0:'no'
    },
    'es':{
        1:'sí',
        0:'no'
    }
}

l=['es','en']

def get_input(i,l):
    return hateval_t[l].format(s=i)

def get_output(o,l):
    return hateval_v[l][o]

In [102]:
# train set + dev set
re_label_train_es=pd.read_csv('data/raw/hateval2019/hateval2019-relabeled-main/data/hateval2019_es_train.csv')
old_label_train_es=pd.read_csv('data/raw/hateval2019/original/hateval2019_es_train.csv').drop(columns=['HS','TR','AG'])
df_train_es=re_label_train_es.merge(old_label_train_es,on='id')
df_train_es['language']=['es']*len(df_train_es)
df_train_es['input']=df_train_es.apply(lambda x: get_input(x.text,x.language), axis=1)
df_train_es['output']=df_train_es.apply(lambda x: get_output(x.HS,x.language), axis=1)



re_label_train_en=pd.read_csv('data/raw/hateval2019/hateval2019-relabeled-main/data/hateval2019_en_train.csv')
old_label_train_en=pd.read_csv('data/raw/hateval2019/original/hateval2019_en_train.csv').drop(columns=['HS','TR','AG'])
df_train_en=re_label_train_en.merge(old_label_train_en,on='id')
df_train_en['language']=['en']*len(df_train_en)
df_train_en['input']=df_train_en.apply(lambda x: get_input(x.text,x.language), axis=1)
df_train_en['output']=df_train_en.apply(lambda x: get_output(x.HS,x.language), axis=1)

re_label_dev_es=pd.read_csv('data/raw/hateval2019/hateval2019-relabeled-main/data/hateval2019_es_dev.csv')
old_label_dev_es=pd.read_csv('data/raw/hateval2019/original/hateval2019_es_dev.csv').drop(columns=['HS','TR','AG'])
df_dev_es=re_label_dev_es.merge(old_label_dev_es,on='id')
df_dev_es['language']=['es']*len(df_dev_es)
df_dev_es['input']=df_dev_es.apply(lambda x: get_input(x.text,x.language), axis=1)
df_dev_es['output']=df_dev_es.apply(lambda x: get_output(x.HS,x.language), axis=1)



re_label_dev_en=pd.read_csv('data/raw/hateval2019/hateval2019-relabeled-main/data/hateval2019_en_dev.csv')
old_label_dev_en=pd.read_csv('data/raw/hateval2019/original/hateval2019_en_dev.csv').drop(columns=['HS','TR','AG'])
df_dev_en=re_label_dev_en.merge(old_label_dev_en,on='id')
df_dev_en['language']=['en']*len(df_dev_en)
df_dev_en['input']=df_dev_en.apply(lambda x: get_input(x.text,x.language), axis=1)
df_dev_en['output']=df_dev_en.apply(lambda x: get_output(x.HS,x.language), axis=1)

df_train=pd.concat([df_dev_es,df_train_es,df_dev_en,df_train_en])

df_train.to_csv('data/processed/hateval_train.csv')

In [79]:
# test set
re_label_test_es=pd.read_csv('data/raw/hateval2019/hateval2019-relabeled-main/data/hateval2019_es_test.csv')
old_label_test_es=pd.read_csv('data/raw/hateval2019/original/hateval2019_es_test.csv').drop(columns=['HS','TR','AG'])
df_test_es=re_label_test_es.merge(old_label_test_es,on='id')
df_test_es['language']=['es']*len(df_test_es)
df_test_es['input']=df_test_es.apply(lambda x: get_input(x.text,x.language), axis=1)
df_test_es['output']=df_test_es.apply(lambda x: get_output(x.HS,x.language), axis=1)



re_label_test_en=pd.read_csv('data/raw/hateval2019/hateval2019-relabeled-main/data/hateval2019_en_test.csv')
old_label_test_en=pd.read_csv('data/raw/hateval2019/original/hateval2019_en_test.csv').drop(columns=['HS','TR','AG'])
df_test_en=re_label_test_en.merge(old_label_test_en,on='id')
df_test_en['language']=['en']*len(df_test_en)
df_test_en['input']=df_test_en.apply(lambda x: get_input(x.text,x.language), axis=1)
df_test_en['output']=df_test_en.apply(lambda x: get_output(x.HS,x.language), axis=1)
df_test=pd.concat([df_test_es,df_test_en])

df_test.to_csv('data/processed/hateval_test.csv')

# CLS

Accessto cls can be found through [link](https://webis.de/data/webis-cls-10.html). Savethe dataset in the folder data/raw

In [ ]:
langs=['en','de','fr','jp']
topics=['books','dvd','music']
df_train=pd.DataFrame()
df_test=pd.DataFrame()

for l in langs:
    for t in topics:
        df1=pd.read_xml(f'data/raw/cls-acl10-unprocessed/{l}/{t}/train.review')
        df2=pd.read_xml(f'data/raw/cls-acl10-unprocessed/{l}/{t}/test.review')
        
        df1['topic_number']=[t]*len(df1)
        df2['topic_number']=[t]*len(df2)

        df1['language']=[l]*len(df1)
        df2['language']=[l]*len(df2)

        

        df_train=pd.concat([df_train,df1])
        df_test=pd.concat([df_test,df2])

In [ ]:
cls_v={
    'en':{
        'negative':'bad',
        'positive':'good'
    },
    'fr':{
        'negative':'mal',
        'positive':'bien'
    },
    'jp':{
        'negative':'悪い',
        'positive':'良い'
    },
    'de':{
        'negative':'Schlecht',
        'positive':'gut'
    }
}

def verbiliser(l,i):
    if i in [1.0,2.0]:
        return cls_v[l]['negative']
    elif i in [4.0,5.0]:
        return cls_v[l]['positive']
    else:
        return
df_train['output']=df_train.apply(lambda x: verbiliser(x['language'],x['rating']),axis=1)
df_test['output']=df_test.apply(lambda x: verbiliser(x['language'],x['rating']),axis=1)

In [ ]:
cls_temp={
    'fr':{
        'Review':'Examen: ',
        'Rating':' Évaluation:'

    },
    'en':{
        'Review':'Review: ',
        'Rating':' Rating:'

    },
    'jp':{
        'Review':'レビュー: ',
        'Rating':' 評価:'
    },
    'de':{
        'Review':'Rezension: ',
        'Rating':'Bewertung:'

    }
}

def temp(l,s):
    return cls_temp[l]['Review']+str(s)+cls_temp[l]['Rating']
  
df_train['input']=df_train.apply(lambda x: temp(x['language'],x['text']),axis=1)
df_train.to_csv('cls_train.csv')

df_test['input']=df_test.apply(lambda x: temp(x['language'],x['text']),axis=1)
df_test.to_csv('cls_test.csv')
